In [ ]:
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import _pickle as pickle

import pycharge as pc
import meshplots as mp

lats = ['square','triangle','rtriangle']

# Code

The repository contains 4 python files:
1. `pycharge.py`:  The main module. Performs calculations such as the interaction matrices, linear response deformation, the young modulus, critical strain at instability, unstable modes, etc.
2. `charge_fields.py`: An auxiliary module that calculates the elastic fields of a charges.
3. `meshplots.py`: A module to draw nice figures of the metamaterial. See below how to recreate Figures 4,5 from the manuscript.
3. `utils.py`: A module with some auxiliary functions

# Data

The fully nonlinear ABAQUS calculations are at `data/FE/fullfe/`. The raw  data  for the charges-formalism calculations is at `data/charges/` and it is used to compute all the results in the paper, as described below.

First, calculate interaction matrices of all lattices and all porosities (should take a minute or two):

In [ ]:
dss = {lat: pc.process_lattice(lat, layers_to_keep=1) for lat in lats}

Calculate the linear response (should take a second or two): 

In [ ]:
lrs = {lat: xr.concat([pc.linear_response(ds.isel(porosity=i)) for i in range(len(ds.porosity))],
          'porosity')
       for lat, ds in dss.items()}
pickle.dump(lrs, open('pkls/linear_response.pkl','wb'))

Calculate non linear response. Should less than a minute.

In [ ]:
nl = {lat: xr.concat([pc.nl_response(ds.isel(porosity=i), n_modes=100)
                      for i in range(len(ds.porosity))], 'porosity')
       for lat, ds in lrs.items()}
pickle.dump(nl, open('pkls/non_linear.pkl','wb'))

#  Visualization

All values are computed on an FE grid, which is a pain to visualiize. To plot spatial fields we first need to triangulate. This is done once by running the following function (takes about a minute):

In [ ]:
mp.calculate_and_save_tris(return_result=False)

In [ ]:
%matplotlib inline
f,_,_ = mp.linear_figure(p=0.5)

In [ ]:
f,_,_ = mp.instability_figure(p=.7)